In [1]:
import numpy as np
import random
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

from time import time
from torchvision import datasets, transforms

from google.colab import drive
drive.mount('/content/drive',force_remount=True)

device = torch.device('cuda')

import sys
sys.path.append("/content/drive/My Drive/LCNN/")
#from LCNN import *

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
! pwd

/content


In [ ]:
def findlayer(model,layernumber,wob):
    if wob==0:
        layer='fc'+str(layernumber)+'.weight'
    else:
        layer='fc'+str(layernumber)+'.bias'
    i=0
    for name in model.state_dict():
        if name==layer:
            break
        i=i+1
    if i==len(model.state_dict()):
        raise IOError('Layer Name Error')
    return i


In [ ]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [ ]:
def BetaAdaptive(model,ln,i,beta=1.0):
    with torch.no_grad():
        firstweight=findlayer(model,ln,0)
        firstbias=findlayer(model,ln,1)
        secondweight=findlayer(model,ln+1,0)
        params = list(model.named_parameters())
        params[firstweight][1].data[i,:]=params[firstweight][1][i,:].data/beta
        params[firstbias][1].data[i]=params[firstbias][1].data[i]/beta
        params[secondweight][1].data[:,i]=params[secondweight][1].data[:,i]*beta

In [ ]:
def adjust(model,images,threshold_u=100.0,threshold_l=0.1,scale=1.0,ln=1, random.shuffle()):
    with torch.no_grad():
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
        L=len(images)
        hs=len(model.state_dict()['fc'+str(ln)+'.weight'])
        d_=torch.zeros(hs).to(device)
        for i in range(hs):
            d=torch.tensor(0)
            for k in range(L):
                image=images[k]
                l=-model.state_dict()['fc'+str(ln)+'.weight'][i]/model.state_dict()['fc'+str(ln)+'.bias'][i]
                x_k=torch.tensor(1.0)/image.to(device)

                d_k=(torch.abs(torch.sum(l*x_k)-torch.tensor(1.0))/torch.norm(x_k))
                d=d+d_k

            d_[i]=d/L
        d_=d_*scale
        d_=d_**2
        d_=torch.min(d_,torch.tensor(threshold_u*1.0))
        d_=torch.max(d_,torch.tensor(threshold_l*1.0))
        
        print(d_)
        for i in range(hs):
            BetaAdaptive(model,ln,i,d_[i])
        print ('Adjusting Layer {}, Kernel Nodes: {}, Adptive Nodes{}' .format(ln, torch.sum((d_<1).int()).item(), torch.sum((d_>1).int()).item()))



In [ ]:
# Device configuration
device = torch.device('cuda')
# Hyper-parameters 
input_size = 784
hidden_size1 = 1000

hidden_size2 = 100

hidden_size3 = 100

num_classes = 10
batch_size = 64


class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size1,hidden_size2,hidden_size3,num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1) 
        self.hidden_size1 = hidden_size1
        #self.fc1.weight.detach().normal_(0.0, 0.1)
        #self.fc1.bias.detach().normal_(0.0, 0.1)
        
        
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2,bias=False)
        #self.fc2.weight.detach().uniform_(-1,1)
        
        
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.relu2 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size3, num_classes,bias=False)
        
        self.lsm=nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        
        
        out = self.fc3(out)
        out = self.relu2(out)
        out = self.fc4(out)
        
        out = self.lsm(out)
        return out


In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
batchsize=64

trainset = datasets.MNIST('/content/drive/My Drive/data', download=True, train=True, transform=transform)
valset = datasets.MNIST('/content/drive/My Drive/data', download=True, train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batchsize, shuffle=False)
valloader = torch.utils.data.DataLoader(valset, batch_size=batchsize, shuffle=False)

In [ ]:
learning_rate = 0.001
torch.manual_seed(0)
model = NeuralNet(input_size, hidden_size1,hidden_size2,hidden_size3,num_classes).to(device)
ad=True
if ad:
    dataiter = iter(train_loader)
    images, labels = dataiter.next()
    images = images.view(images.shape[0], -1)
    #adjust(model,images,threshold_u=1,threshold_l=0.01,scale=0.8)
    adjust(model,images,threshold_u=10,threshold_l=0.01,scale=0.8,ln=1)
    #adjust(model,images,threshold_u=10,threshold_l=1,scale=0.8)
    
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate,momentum=0.9)
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.NLLLoss()
num_epochs=700
dev=100
losslist=[]


totalloss=[]



model.fc3.register_forward_hook(get_activation('fc2'))


count=0

for epoch in range(num_epochs):
    loss_epoch=0
    for i, (images, labels) in enumerate(train_loader):  
        

        images = images.view(images.shape[0], -1).to(device)
        
        #print(images,labels)
        # Forward pass
        outputs = model(images).to(device)
        loss = criterion(outputs, labels).to(device)
        
        
        totalloss.append(loss.item())

        loss_epoch=loss_epoch+loss.item()
        print(count,loss.item())
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        count=count+1
        if count==50:
            output = model(images)
            adjust(model,activation['fc2'],threshold_u=10,threshold_l=0.01,scale=0.8,ln=3)
            
        if count>500:
            break
        
    if count>500:
        break
                
        
        
    loss_epoch=loss_epoch/len(train_loader) 
    losslist.append(loss_epoch)
    if loss_epoch<0.02:
        break
    print ('Epoch [{}/{}], Loss: {:.4f}' .format(epoch+1, num_epochs, loss_epoch))


tensor([ 0.1820,  0.6269,  0.3382,  0.3390,  1.2558,  1.7743,  2.2847,  7.3919,
         0.7454,  0.8111,  0.1785,  0.3652,  0.3134,  0.2691,  0.2367,  0.1591,
         0.1568,  0.2558,  0.3016,  1.3846,  3.2755,  0.5948,  1.0742,  0.3449,
         0.7438,  0.1079,  1.4494,  0.4824,  3.2023,  0.3570,  0.3684,  0.2019,
         0.9853,  0.1933, 10.0000,  0.2131,  0.5786,  0.5468,  0.3057,  0.1829,
         0.8437, 10.0000,  1.3770,  1.2686,  0.2292,  0.5187,  0.8510,  0.4160,
         0.2582, 10.0000,  7.0040,  0.3385,  6.1540,  0.4186,  8.4819,  2.2911,
         3.5933, 10.0000,  0.5284,  9.2418,  7.8243,  0.2789,  0.5384,  0.5270,
        10.0000,  3.4224,  0.1195, 10.0000,  1.1683,  0.9502,  0.5592,  2.5515,
         0.4614,  8.6381,  0.2665,  2.9422,  2.8777,  0.7278,  2.4429, 10.0000,
         0.3681,  0.6449,  4.9783,  0.4366,  1.3035,  0.2838,  2.5326,  6.2872,
         0.2801,  0.3079,  0.1810,  1.7211,  0.2595,  2.9894,  0.3975, 10.0000,
         3.3330,  0.4503,  0.2841,  0.27

In [ ]:
torch.set_default_tensor_type('torch.cuda.FloatTensor')
L=len(images)
ln=1
hs=len(model.state_dict()['fc'+str(ln)+'.weight'])
d_=torch.zeros(hs).to(device)
for i in range(hs):
    d=torch.tensor(0)
    for k in range(L):
        image=images[k]
        l=-model.state_dict()['fc'+str(ln)+'.bias'][i]/model.state_dict()['fc'+str(ln)+'.weight'][i]
        x_k=torch.tensor(1.0)/image.to(device)
        d_k=(torch.abs(torch.sum(l*x_k)-torch.tensor(1.0))/torch.norm(x_k))
        
        d=d+d_k
    input("Press Enter to continue...")
    print(d/L)
    d_[i]=d/L

Press Enter to continue...
tensor(26.0358)
Press Enter to continue...
tensor(19.6310)
Press Enter to continue...
tensor(796.5876)
Press Enter to continue...
tensor(14.5831)
Press Enter to continue...
tensor(7.1308)
Press Enter to continue...
tensor(9.1515)
Press Enter to continue...
tensor(0.7501)
Press Enter to continue...
tensor(0.7818)
Press Enter to continue...
tensor(3.0048)


KeyboardInterrupt: ignored

In [ ]:
model.state_dict()['fc'+str(ln)+'.weight'][0][0]

tensor(-0.0007)

In [ ]:
b= torch.sum((a>0).int())

In [ ]:
b.item()

3

In [ ]:
model.state_dict()['fc'+str(ln)+'.weight'][1][0]

tensor(-0.0024)

In [ ]:
import random

In [ ]:
random.shuffle(range(10))

TypeError: ignored

In [ ]:
a=np.arange(10)

In [ ]:
print(random.shuffle(a))

None


In [ ]:
a

array([3, 8, 6, 9, 0, 2, 7, 1, 4, 5])